In [17]:
import requests
import pandas as pd
import re

def GetAllFund():
    '''
        功能：获取所有基金名称、代码、类型，天天基金数据接口：http://fund.eastmoney.com/js/fundcode_search.js
        传入：无
        返回：所有基金基础数据
    '''
    url = "http://fund.eastmoney.com/js/fundcode_search.js"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
    }
    res = requests.get(url,headers=headers)
    res.encoding = "utf-8"
    list_ = eval(re.findall(r'\[.*\]',res.text)[0])
    df = pd.DataFrame(list_)
    df.columns = ['基金代码','基金拼音简写','基金名称','基金类型','基金拼音全称']
    
    return df

funds = GetAllFund()
funds

,基金代码,基金拼音简写,基金名称,基金类型,基金拼音全称
0,000001,HXCZHH,华夏成长混合,混合型-灵活,HUAXIACHENGZHANGHUNHE
1,000002,HXCZHH,华夏成长混合(后端),混合型-灵活,HUAXIACHENGZHANGHUNHE
2,000003,ZHKZZZQA,中海可转债债券A,债券型-可转债,ZHONGHAIKEZHUANZHAIZHAIQUANA
3,000004,ZHKZZZQC,中海可转债债券C,债券型-可转债,ZHONGHAIKEZHUANZHAIZHAIQUANC
4,000005,JSZQXYDQZQ,嘉实增强信用定期债券,债券型-长债,JIASHIZENGQIANGXINYONGDINGQIZHAIQUAN
...,...,...,...,...,...
18009,970205,XZZGJQLXXZLLGYCYQZQC,兴证资管金麒麟兴享增利六个月持有期债券C,债券型-混合债,XINGZHENGZIGUANJINQILINXINGXIANGZENGLILIUGEYUE...
18010,970206,ZJYSLHYNCYHHC,中金优势领航一年持有混合C,混合型-偏股,ZHONGJINYOUSHILINGHANGYINIANCHIYOUHUNHEC
18011,970207,GXRFZQC,国信睿丰债券C,债券型-混合债,GUOXINRUIFENGZHAIQUANC
18012,970208,GXJDZHSGYCYHHFOF,国信经典组合三个月持有混合(FOF),FOF,GUOXINJINGDIANZUHESANGEYUECHIYOUHUNHEFOF


In [5]:
funds['基金类型'].unique()

array(['混合型-灵活', '债券型-可转债', '债券型-长债', '混合型-偏股', '指数型-股票', '货币型',
       '债券型-混合债', 'QDII', '股票型', '债券型-中短债', '混合型-偏债', '商品（不含QDII）',
       '混合-绝对收益', '理财型', '混合型-平衡', 'FOF', 'Reits'], dtype=object)

In [12]:
funds[funds['基金名称'].str.contains('ETF')]

,基金代码,基金拼音简写,基金名称,基金类型,基金拼音全称
6,000008,JSZZ500ETFLJA,嘉实中证500ETF联接A,指数型-股票,JIASHIZHONGZHENG500ETFLIANJIEA
37,000051,HXHS300ETFLJA,华夏沪深300ETF联接A,指数型-股票,HUAXIAHUSHEN300ETFLIANJIEA
40,000055,GFNZ100ETFLJMYQDIIA,广发纳指100ETF联接美元(QDII)A,QDII,GUANGFANAZHI100ETFLIANJIEMEIYUANQDIIA
54,000071,HXHSETFLJA,华夏恒生ETF联接A,QDII,HUAXIAHENGSHENGETFLIANJIEA
58,000075,HXHSETFLJXH,华夏恒生ETF联接现汇,QDII,HUAXIAHENGSHENGETFLIANJIEXIANHUI
...,...,...,...,...,...
17416,588370,NFSZKCB50CFZQCLETF,南方上证科创板50成份增强策略ETF,指数型-股票,NANFANGSHANGZHENGKECHUANGBAN50CHENGFENZENGQIAN...
17417,588380,FGZZKCCY50ETF,富国中证科创创业50ETF,指数型-股票,FUGUOZHONGZHENGKECHUANGCHUANGYE50ETF
17418,588390,BSZZKCCY50ETF,博时中证科创创业50ETF,指数型-股票,BOSHIZHONGZHENGKECHUANGCHUANGYE50ETF
17419,588400,JSZZKCCY50ETF,嘉实中证科创创业50ETF,指数型-股票,JIASHIZHONGZHENGKECHUANGCHUANGYE50ETF


In [13]:
# 获取基金详细信息


import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
from functools import partial 

def Get_html(fund_code,start_date,end_date,type_="lsjz",page=1,per=20):
    '''
        获取基金网页数据
    '''
    url = "http://fund.eastmoney.com/f10/F10DataApi.aspx?type={}&code={}&page={}&sdate={}&edate={}&per={}"\
            .format(type_,fund_code,page,start_date,end_date,per)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
    }
    HTML = requests.get(url,headers=headers)
    HTML.encoding = "utf-8"
    
    return HTML

def Get_pages(HTML):
    '''
        获取最大页数
    '''
    pages = re.findall(r'pages:(.*),',HTML.text)[0]
    
    return int(pages)

def Get_FundData_history(HTML):
    '''
        通过html获取基金历史数据
    '''
    soup = BeautifulSoup(HTML.text, 'html.parser')
    trs = soup.find_all("tr")
    res = []
    for tr in trs[1:]:
        date = tr.find_all("td")[0].text # 净值日期
        unit_net = tr.find_all("td")[1].text # 单位净值
        acc_net = tr.find_all("td")[2].text # 累计净值
        fund_r = tr.find_all("td")[3].text # 日增长率
        buy_status = tr.find_all("td")[4].text # 申购状态
        sell_status = tr.find_all("td")[5].text # 赎回状态
        res.append([date,unit_net,acc_net,fund_r,buy_status,sell_status])
    df = pd.DataFrame(res,columns=['净值日期','单位净值','累计净值','日增长率','申购状态','赎回状态'])
    
    return df

def Get_FundData_main(fund_code,start_date,end_date):
    '''
        获取基金数据主函数（仅支持单基金）
    '''
    html = Get_html(fund_code,start_date,end_date)
    pages = Get_pages(html)
    res_df = pd.DataFrame()
    for page in range(1,pages+1):
        html = Get_html(fund_code,start_date,end_date,"lsjz",page)
        df_ = Get_FundData_history(html)
        res_df = pd.concat([res_df,df_])
        
    res_df.insert(0,"基金代码",fund_code)
        
    return res_df
    
    


In [14]:

start_date = "2022-01-01"
end_date = "2022-12-30"
codes = ['320007','161725','012723'] # 基金代码列表
fund_df = pd.concat(list(map(partial(Get_FundData_main,start_date=start_date,end_date=end_date),codes))) #结合map方法获取多基金数据
fund_df

,基金代码,净值日期,单位净值,累计净值,日增长率,申购状态,赎回状态
0,320007,2022-12-30,1.2710,1.7160,-0.63%,开放申购,开放赎回
1,320007,2022-12-29,1.2790,1.7240,0.08%,开放申购,开放赎回
2,320007,2022-12-28,1.2780,1.7230,-1.39%,开放申购,开放赎回
3,320007,2022-12-27,1.2960,1.7410,0.93%,开放申购,开放赎回
4,320007,2022-12-26,1.2840,1.7290,2.88%,开放申购,开放赎回
...,...,...,...,...,...,...,...
17,012723,2022-01-10,1.0022,1.0022,-1.15%,开放申购,开放赎回
18,012723,2022-01-07,1.0139,1.0139,-1.88%,开放申购,开放赎回
19,012723,2022-01-06,1.0333,1.0333,0.11%,开放申购,开放赎回
0,012723,2022-01-05,1.0322,1.0322,-3.50%,开放申购,开放赎回


In [15]:
fund_df.reverse()

AttributeError: 'DataFrame' object has no attribute 'reverse'